In [ ]:
!pip install openai==0.28
!pip install line-bot-sdk
!pip install flask-ngrok
!pip install flask
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 853.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.4/758.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 7.8 MB/s eta 0:00:00


In [ ]:
from flask import Flask, request, abort
from pyngrok import ngrok, conf
from flask_ngrok import run_with_ngrok
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
import openai

openai.api_key = 'sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc'

conf.get_default().auth_token = "2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz"

app = Flask(__name__)
run_with_ngrok(app)

line_bot_api = LineBotApi('Lx65hwBaeVtUtyz4DLeWXGke+1Np3yUyBQ0yPh3LK9DETzVXHp3LaBUHIOVYR0K5/3+skVNUXR9h9Mp2ouYZGMmhgAJQ/6fvYU3kCUhfnp+FrOt43YDSSXwff9h2m5SMPdPiSddnoLwYIzHxYqWgIQdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('e0069f76b523e833fae449753218f7e1')

# 質問回数を追跡するための変数
user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    # 文字数チェック
    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            # 回答の文字数をチェックして制限
            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    public_url = ngrok.connect(5000)
    app.run()


<ipython-input-2-2c087363a365>:16: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi('Lx65hwBaeVtUtyz4DLeWXGke+1Np3yUyBQ0yPh3LK9DETzVXHp3LaBUHIOVYR0K5/3+skVNUXR9h9Mp2ouYZGMmhgAJQ/6fvYU3kCUhfnp+FrOt43YDSSXwff9h2m5SMPdPiSddnoLwYIzHxYqWgIQdB04t89/1O/w1cDnyilFU=')
<ipython-input-2-2c087363a365>:17: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler('e0069f76b523e833fae449753218f7e1')


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://bff1-34-69-34-112.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


<ipython-input-2-2c087363a365>:72: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(
INFO:werkzeug:127.0.0.1 - - [27/Jul/2024 03:35:38] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Jul/2024 05:09:03] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Jul/2024 05:10:43] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Jul/2024 05:11:08] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Jul/2024 05:11:30] "POST / HTTP/1.1" 200 -
